In [1]:
import ccxt

# Set up Binance API
binance = ccxt.binance()
symbol = "BTC/USDT"


In [2]:
import import_ipynb
from predictor import XGBoostPredictor, RNNPredictor, LSTMPredictor

import numpy as np


importing Jupyter notebook from predictor.ipynb


In [3]:
rnn = RNNPredictor("../models/rnn_model.h5")
lstm = LSTMPredictor("../models/lstm_model.h5")
xgb = XGBoostPredictor("../models/xgboost_model.json")


def predict(inputs, model_name, n_to_predict):
    if model_name == "RNN":
        model = rnn
    elif model_name == "LSTM":
        model = lstm
    elif model_name == "XGBoost":
        model = xgb

    return model.predict(inputs, n_to_predict)


In [4]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input

import plotly.graph_objs as go

import dash_bootstrap_components as dbc


C:\Users\Admin\AppData\Local\Temp\ipykernel_20244\1019621102.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\Admin\AppData\Local\Temp\ipykernel_20244\1019621102.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [5]:
dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, dbc_css])

app.layout = html.Div(
    [
        html.H4(
            "BTC-USDT Analysis Dashboard",
            className="bg-primary text-white p-2 mb-2 text-center",
        ),
        dbc.Card(
            [
                html.Div(
                    [
                        dbc.Label("Select model"),
                        dcc.Dropdown(
                            id="model-name",
                            options=[
                                {"label": "LSTM", "value": "LSTM"},
                                {"label": "RNN", "value": "RNN"},
                                {
                                    "label": "XGBoost",
                                    "value": "XGBoost",
                                },
                            ],
                            value="XGBoost",
                            clearable=False,
                        ),
                        dbc.Label("Select features extraction"),
                        dbc.Checklist(
                            id="features",
                            options=[
                                "Close",
                                "Price of Change",
                                "RSI",
                                "Bolling Bands",
                                "Moving Average",
                            ],
                            value="Close",
                            inline=True,
                        ),
                    ]
                ),
            ],
            body=True,
            style={
                "margin": "2rem 1rem 0rem",
            },
        ),
        dbc.Card(
            dbc.Tabs([dbc.Tab([dcc.Graph(id="live-graph")], label="Chart")]),
            style={
                "margin": "2rem 1rem 0rem",
            },
        ),
        html.Div(id="remaining-time"),
        dcc.Interval(
            id="interval-component",
            interval=15 * 1000,  # in milliseconds
            n_intervals=0,
        ),
    ]
)


In [6]:
import pandas as pd


@app.callback(
    [Output("live-graph", "figure"), Output("remaining-time", "children")],
    [Input("interval-component", "n_intervals"), Input("model-name", "value")],
)
def update_graph(n, model_name):
    ohlcv = binance.fetch_ohlcv(symbol, "1h", limit=1000)
    df = pd.DataFrame(
        ohlcv, columns=["timestamp", "open", "high", "low", "close", "volume"]
    )
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")

    input_data = df[["timestamp", "close"]]

    n_to_predict = 100

    predictions = predict(input_data[["timestamp", "close"]], model_name, n_to_predict)
    predictions = list(np.concatenate(predictions))

    trace = go.Candlestick(
        x=df["timestamp"],
        open=df["open"],
        high=df["high"],
        low=df["low"],
        close=df["close"],
        name=symbol,
    )

    trace_predictions = go.Scatter(
        x=df["timestamp"].iloc[-n_to_predict:],
        y=predictions,
        mode="lines",
        name="Predicted Close",
        line=dict(color="orange"),
    )

    layout = go.Layout(
        title=f"Live OHLCV Chart - {symbol}",
        xaxis=dict(title="Time"),
        yaxis=dict(title="Price"),
        showlegend=True,
        height=700,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    )

    remaining_time = 10 - (n % 10)  # Calculate remaining seconds

    return {
        "data": [trace, trace_predictions],
        "layout": layout,
    }, f"input: {model_name}"


# Start the app.

In [7]:
import webbrowser

if __name__ == "__main__":
    # webbrowser.open('http://127.0.0.1:8050/', new=0, autoraise=True)
    app.run_server(debug=False)


XGBoost
XGBoost
XGBoost
XGBoost
